In [125]:
# Aplicação completa:
# Esta aplicação faz uso do tokenizador BPE (Byte Pair Encoding) para lidar com textos e realizar a tokenização de forma eficiente, especialmente útil para sistemas NLP.

In [127]:
# Importa as bibliotecas necessárias para a aplicação:

In [129]:
import os  # Para manipulação de diretórios e arquivos.
import json  # Para carregar e manipular dados em formato JSON.
from tqdm import tqdm  # Exibe uma barra de progresso durante operações demoradas.
import sys  # Permite manipular parâmetros e caminhos do sistema.
from collections import defaultdict

In [131]:
# Define o caminho para a pasta 'corpus', que deve conter os arquivos JSON a serem processados.

In [133]:
pasta_corpus = r'C:\Users\tagsa\Downloads\corpus'  # O 'r' antes da string indica uma raw string, preservando as barras invertidas.

In [214]:
# Carrega e combina todos os textos dos arquivos JSON em uma única string.
# Exibe no console os resultados do processamento, fornecendo informações sobre o corpus carregado.

In [215]:
corpus_texto = carregar_textos_json(pasta_corpus)

print(f"Número de arquivos JSON válidos lidos: {arquivos_validos}")  # Exibe o número de arquivos JSON válidos lidos.
print(f"Corpus carregado com {sum(len(texto) for texto in textos)} caracteres.")  # Exibe o tamanho do corpus, ou seja, a quantidade total de caracteres nos textos carregados.
print(f"Número de textos individuais: {textos_encontrados}")  # Exibe o número total de textos individuais extraídos dos arquivos JSON.

Número de arquivos JSON encontrados: 10000


Carregando arquivos JSON: 100%|████████████████████████████████████████████| 10000/10000 [00:15<00:00, 629.30arquivo/s]


Número de arquivos JSON válidos lidos: 10000
Corpus carregado com 71023261 caracteres.
Número de textos individuais: 10000


In [143]:
# Adiciona ao sys.path o diretório onde o módulo BPE_Tokenizer pode estar localizado.

In [145]:
sys.path.append(r"C:\Users\tagsa\Downloads")  # Use raw string para evitar erros com barras invertidas.

In [147]:
# Importa o tokenizador BPE do módulo localizado no caminho adicionado.

In [149]:
from bpe_tokenizer_otimizado_e_comentado import BPE_Tokenizer

In [151]:
# Inicializa o tokenizador BPE.

In [153]:
tokenizer = BPE_Tokenizer()

In [155]:
# Esta função realiza a leitura de arquivos JSON armazenados em uma pasta específica.
# O objetivo é extrair textos desses arquivos e calcular quantos textos e arquivos foram processados com sucesso.

In [193]:
def carregar_textos_json(pasta):
    """
    Carrega e combina todos os textos dos arquivos JSON em um único texto.

    Parâmetros:
    -----------
    pasta : str
        Caminho da pasta onde estão localizados os arquivos JSON.

    Retorna:
    --------
    str:
        Todo o conteúdo dos arquivos JSON combinados em uma única string.
    """
    corpus = []

    if not os.path.exists(pasta):
        print(f"Pasta não encontrada: {pasta}")
        return ""

    arquivos = [arq for arq in os.listdir(pasta) if arq.endswith('.json')]
    print(f"Número de arquivos JSON encontrados: {len(arquivos)}")

    for arquivo in tqdm(arquivos, desc="Carregando arquivos JSON", unit="arquivo"):
        caminho = os.path.join(pasta, arquivo)
        try:
            with open(caminho, "r", encoding="utf-8") as f:
                data = json.load(f)
            if 'text' in data:
                corpus.append(data['text'])
        except (json.JSONDecodeError, OSError) as e:
            print(f"Erro ao processar {arquivo}: {str(e)}")

    return " ".join(corpus)  # Combina todos os textos em uma única string.

def get_most_frequent_pairs(ids):
    """
    Calcula a frequência de pares consecutivos de tokens na lista de IDs.
    """
    pairs = defaultdict(int)
    for i in range(len(ids) - 1):
        pair = (ids[i], ids[i + 1])
        pairs[pair] += 1
    return pairs

def mesclar_pares(ids, tokenizer, merges, limite=20):
    """
    Realiza a mesclagem de no máximo 20 pares de tokens mais frequentes.
    """
    stats = tokenizer.get_stats(ids)

    # Ordena os pares pela frequência em ordem decrescente.
    pares_ordenados = sorted(stats.items(), key=lambda x: x[1], reverse=True)[:limite]

    for idx, (pair, freq) in enumerate(pares_ordenados):
        token_a = tokenizer.vocab.get(pair[0], b'?').decode('utf-8', errors='replace')
        token_b = tokenizer.vocab.get(pair[1], b'?').decode('utf-8', errors='replace')

        print(f"Mesclando {pair} ({token_a}, {token_b}) em um novo token {256 + idx} com frequência {freq}")

        ids = tokenizer.merge(ids, pair, 256 + idx)
        merges[pair] = (256 + idx, freq)

    return ids

In [195]:
# Treinamento do tokenizador BPE.

In [197]:
print("Treinando o tokenizador BPE...")
tokenizer.train(corpus_texto, vocab_size=276)

Treinando o tokenizador BPE...


Treinamento do tokenizador: 100%|████████████████████████████████████████████████| 20/20 [07:08<00:00, 21.41s/iteração]

Treinamento do tokenizador BPE concluído!


In [198]:
# Inicializa o dicionário de merges.

In [199]:
merges = {}

In [200]:
# Converte o corpus para uma lista de IDs.

In [201]:
ids = list(corpus_texto.encode("utf-8"))

In [202]:
# Realiza a mesclagem dos 20 pares mais frequentes (conforme definido).

In [203]:
ids = mesclar_pares(ids, tokenizer, merges)

Mesclando (111, 32) (o,  ) em um novo token 256 com frequência 1934443
Mesclando (97, 32) (a,  ) em um novo token 257 com frequência 1767265
Mesclando (101, 32) (e,  ) em um novo token 258 com frequência 1690522
Mesclando (115, 32) (s,  ) em um novo token 259 com frequência 1397720
Mesclando (32, 100) ( , d) em um novo token 260 com frequência 1372702
Mesclando (100, 101) (d, e) em um novo token 261 com frequência 1069531
Mesclando (32, 101) ( , e) em um novo token 262 com frequência 831530
Mesclando (44, 32) (,,  ) em um novo token 263 com frequência 799119
Mesclando (114, 97) (r, a) em um novo token 264 com frequência 768678
Mesclando (101, 115) (e, s) em um novo token 265 com frequência 768076
Mesclando (32, 97) ( , a) em um novo token 266 com frequência 765466
Mesclando (100, 111) (d, o) em um novo token 267 com frequência 736397
Mesclando (111, 115) (o, s) em um novo token 268 com frequência 683013
Mesclando (32, 112) ( , p) em um novo token 269 com frequência 658197
Mesclando (97

In [204]:
# Exibe estatísticas finais.

In [205]:
print(f"\nComprimento final dos tokens: {len(corpus_texto)}")
print(f"Comprimento final dos IDs: {len(ids)}")


Comprimento final dos tokens: 71033260
Comprimento final dos IDs: 60952296


In [206]:
# Calcula e exibe a taxa de compressão.

In [207]:
taxa_compressao = len(corpus_texto) / len(ids)
print(f"Taxa de compressão: {taxa_compressao:.2f}X")

Taxa de compressão: 1.17X
